In [1]:
import os
os.chdir('../code/numbertheory')

In [2]:
from utils import *

2+ i3

We want to determine the possible traces of curves with $j = 0,1728$ when those $j$-invariants are ordinary.
Note that once we know one of the traces, obtaining the others is easy.

* Finding an explicit element of norm $p$ in the associated ring
* Finding a pair $a,b$ such that $a^2+b^2 = p$ or $a^2+3b^2 = p$.
* Computing the cardinality of $y^2 = x^3 + fx$ or $y^2 = x^3 +g$ over $\mathbb{F}_p$.

We can always do these using naive methods
that have complexity $\mathcal{O}(p)$.

If we use more advanced tools,
like Schoof's algorithm,
we can obtain the cardinality much faster.

Our goal in this notebook to describe a low tech algorithm that does much better than the methods listed above.

1. Find an integer $r$ that reduces to a primitive cube/fourth root of unity mod $p$.
This is easy - we pick random integers $x$
and compute $x^{(p-1)/n}\pmod p$ until we obtain an element which is not equal to 1 (or -1 if $n = 4$).

2. We look for a pair of integers $x,y$ that satisfy two conditions:
* $x\equiv r y \pmod p$
* The vector $(x,y)$ is as small as possible.
By Minkowski's geometry of numbers argument,
we can show that $x^2+y^2 = p$ or $x^2 + xy + y^2 = p$.
Now, it is easy to obtain a pair that satisfies the first condition - we just take $(1,r)$.
We will use the Euclidean algorithm to obtain a sequence of vectors that continue to satisfy the first condition,
but decrease in length.

In [4]:
testabs = [(a,b) for a in range(-4,17) for b in range(2,9)]
testzs = [GaussianInteger(ab) for ab in testabs]
testzpairs = [(testzs[i],testzs[j]) for i in range(len(testzs)-1) for j in range(i+1,len(testzs))]

In [11]:
def test_mod(pair):
    z1,z2 = pair
    n2 = z2.nrm
    nr = (z1%z2).nrm
    return nr<n2

In [13]:
[ab for ab in testzpairs if not test_mod(ab)]

[]

In [6]:
z0 = GaussianInteger((1,4))
z1 = GaussianInteger((-23,91))
z2 = GaussianInteger((17,23))
z3 = GaussianInteger((2,-8))

In [8]:
z0.conj()

1- i4

In [13]:
def get_rou_mod(m,p):
    m0 = gcd(m,p-1)
    k = (p-1)//m0
    for x in range(2,p-1):
        xk = pow(x,k,p)
        xkg = [pow(xk,i,p) for i in range(m0)]
        if len(set(xkg))== m0:
            return xk
    return 0

def improve_approx_gauss(xy,p):
    x,y = xy
    k = p//(abs(y))
    x1 = (x*k)%p
    x2 = (x*(k+1))%p
    y1 = p-k*abs(y)
    y2 = (k+1)*abs(y)-p
    v1 = (x1,y1)
    v2 = (x2,y2)
    n1 = x1**2+y1**2
    n2 = x2**2+y2**2
    if n1 == p or n1 <= n2:
        return v1
    else:
        return v2

def get_gi_vecs(p):
    if p % 4 == 3:
        return []
    elif p == 2:
        return [(1,1)]
    r0 = min(get_rou_mod(4,p),p-get_rou_mod(4,p))
    if 1 + r0**2 == p:
        return [(1,r0)]
    v0 = (1,p-r0)
    v1 = (1,r0)
    n0 = 1+(p-r0)**2
    n1 = 1+r0**2
    if gcd(n0,n1)==p:
        return [v0,v1]
    v2 = improve_approx_gauss(v1,p)
    n2= v2[0]**2+v2[1]**2
    if n2 == p:
        return [v2]
    return [v0,v1,v2,(n0,n1,n2)]
    

In [ ]:
def get_rou_mod(m,p):
    m0 = gcd(m,p-1)
    k = (p-1)//m0
    for x in range(2,p-1):
        xk = pow(x,k,p)
        xkg = [pow(xk,i,p) for i in range(m0)]
        if len(set(xkg))== m0:
            return xk
    return 0

def improve_approx_gauss(xy,p):
    x,y = xy
    k = p//(abs(y))
    x1 = (x*k)%p
    x2 = (x*(k+1))%p
    y1 = p-k*abs(y)
    y2 = (k+1)*abs(y)-p
    v1 = (x1,y1)
    v2 = (x2,y2)
    n1 = x1**2+y1**2
    n2 = x2**2+y2**2
    if n1 == p or n1 <= n2:
        return v1
    else:
        return v2
    
def sum_of_squares_rep(p):
    if p % 4 == 3:
        return None
    if p == 2:
        return GaussianInteger((1,1))
    # r0 is a square root of -1 mod p
    # We have to choose between r0 and p-r0;
    # We pick whichever is smallest
    r0 = min(get_rou_mod(4,p),p-get_rou_mod(4,p))
    # Our first guess is 1 +r0.
    # If p = 1+n^2, then n is necessarily r0
    z1 = GaussianInteger((1,r0))
    if z1.nrm == p:
        return z1
    # We can construct a second vector using the
    # other square root of -1. This will definitely
    # not work, but it could help us improve first guess.
    z0 = GaussianInteger((1,p-r0))
    while z1.nrm !=p and z1.nrm < z0.nrm:
        if gcd(z0.nrm,z1.nrm)< z1.nrm:
            z01 = gcd_gauss(z0,z1)
            z01c = gcd_gauss(z0,z1.conj())
            if z01.nrm % p == 0:
                if z01.nrm == p:
                    return z01
                z1 = z01
            elif z01c.nrm % p == 0:
                if z01c.nrm == p:
                    return z01c
                z1 = z01c
        if z1.nrm == p:
            return z1
        z2 = GaussianInteger(improve_approx_gauss(z1.vec,p))
        z0 = z1
        z1 = z2
    return z1

In [19]:
primelist1m4 = [p for p in primesBetween(2**10,2**14) if p % 4 == 1]
reps = {p:sum_of_squares_rep(p) for p in primelist1m4}

In [20]:
reps

{1033: -3+ i32,
 1049: -5+ i32,
 1061: 10+ i31,
 1069: 13- i30,
 1093: 2+ i33,
 1097: 16+ i29,
 1109: -22+ i25,
 1117: -21+ i26,
 1129: 27- i20,
 1153: 8+ i33,
 1181: 5- i34,
 1193: 13- i32,
 1201: 25- i24,
 1213: -27+ i22,
 1217: 16- i31,
 1229: 2+ i35,
 1237: -9- i34,
 1249: -15+ i32,
 1277: 11+ i34,
 1289: 8+ i35,
 1297: 1+ i36,
 1301: 26- i25,
 1321: 5+ i36,
 1361: 20- i31,
 1373: 2+ i37,
 1381: -15- i34,
 1409: -25+ i28,
 1429: -30+ i23,
 1433: -8+ i37,
 1453: -3+ i38,
 1481: 16- i35,
 1489: -20+ i33,
 1493: 7- i38,
 1549: 18- i35,
 1553: 23- i32,
 1597: 21- i34,
 1601: 1+ i40,
 1609: 3+ i40,
 1613: 13- i38,
 1621: 10- i39,
 1637: -26+ i31,
 1657: -19- i36,
 1669: -15- i38,
 1693: 18+ i37,
 1697: 4+ i41,
 1709: -22+ i35,
 1721: -11+ i40,
 1733: 17- i38,
 1741: 30- i29,
 1753: -32+ i27,
 1777: 16+ i39,
 1789: -5+ i42,
 1801: -35+ i24,
 1861: 31- i30,
 1873: 28- i33,
 1877: 14- i41,
 1889: -17- i40,
 1901: 35- i26,
 1913: 8+ i43,
 1933: 13- i42,
 1949: 10- i43,
 1973: -23+ i38,
 199

In [18]:
givecs

{1033: -3+ i32,
 1049: -5+ i32,
 1061: 10+ i31,
 1069: 13- i30,
 1093: 2+ i33,
 1097: 16+ i29,
 1109: -22+ i25,
 1117: -21+ i26,
 1129: 27- i20,
 1153: 8+ i33,
 1181: 5- i34,
 1193: 13- i32,
 1201: 25- i24,
 1213: -27+ i22,
 1217: 16- i31,
 1229: 2+ i35,
 1237: -9- i34,
 1249: -15+ i32,
 1277: 11+ i34,
 1289: 8+ i35,
 1297: 1+ i36,
 1301: 26- i25,
 1321: 5+ i36,
 1361: 20- i31,
 1373: 2+ i37,
 1381: -15- i34,
 1409: -25+ i28,
 1429: -30+ i23,
 1433: -8+ i37,
 1453: -3+ i38,
 1481: 16- i35,
 1489: -20+ i33,
 1493: 7- i38,
 1549: 18- i35,
 1553: 23- i32,
 1597: 21- i34,
 1601: 1+ i40,
 1609: 3+ i40,
 1613: 13- i38,
 1621: 10- i39,
 1637: -26+ i31,
 1657: -19- i36,
 1669: -15- i38,
 1693: 18+ i37,
 1697: 4+ i41,
 1709: -22+ i35,
 1721: -11+ i40,
 1733: 17- i38,
 1741: 30- i29,
 1753: -32+ i27,
 1777: 16+ i39,
 1789: -5+ i42,
 1801: -35+ i24,
 1861: 31- i30,
 1873: 28- i33,
 1877: 14- i41,
 1889: -17- i40,
 1901: 35- i26,
 1913: 8+ i43,
 1933: 13- i42,
 1949: 10- i43,
 1973: -23+ i38,
 199

In [ ]:
def test_hypothesis(p):
    r0 = get_rou_mod(4,p)
    r1 = max(r0,p-r0)
    r2 = p-r1
    x = smallest_q(r2,p)
    y = smallest_rem(r2,p)
    n1 = 1+r1**2
    n2 = 1+ r2**2
    n3 = x**2 + y**2
    return [(n1,n2,n3),max(n1%p,n2%p,n3%p),(n1//p,n2//p,n3//p),(gcd(n1,n2)//p,gcd(n2,n3)//p)]   

oneshots = {}
rawdata1m4 = {}
for p in primelist1m4:
    r = get_rou_mod(4,p)
    r = min(r,p-r)
    if r**2+1==p:
        oneshots[p]=(1,r)
    else:
        rawdata1m4[p]=test_hypothesis(p)

expected = []
unexpected = []
for p in rawdata1m4:
    if min(rawdata1m4[p][-1])<=1:
        expected.append(p)
    else:
        unexpected.append(p)

In [4]:
ptestbig = [p for p in primesBetween(2**10,2**25) if p % 4 == 1]

In [5]:
psample = [ptestbig[2**k] for k in range(20)]

In [6]:
reps = {p:sum_of_squares_rep(p) for p in psample}

In [7]:
{p-reps[p].nrm for p in reps}

{0}

In [8]:
len(bin(len(ptestbig)))

22

In [9]:
reps

{1049: -5+ i32,
 1061: 10+ i31,
 1093: 2+ i33,
 1129: 27- i20,
 1237: -9- i34,
 1493: 7- i38,
 2017: 9- i44,
 3001: -20+ i51,
 5233: -7+ i72,
 9721: 64- i75,
 19709: 53- i130,
 41117: 59+ i194,
 86257: 216- i199,
 182921: 164+ i395,
 388897: -239- i576,
 825049: 768- i485,
 1745753: -448+ i1243,
 3685973: 857- i1718,
 7760213: -1082+ i2567,
 16298153: -28+ i4037}

In [179]:
def confirm_order(x,m,p):
    if pow(x,m,p)!=1:
        return False
    for k in range(1,m):
        if m%k==0:
            if pow(x,k,p)==1:
                return False
    return True

def find_gen_multgrp_orderm(m,p):
    if (p-1)%m !=0:
        return None
    k = (p-1)//m
    for x in range(2,p):
        xk = pow(x,k,p)
        if confirm_order(xk,m,p):
            return min(xk,p - xk)
    return None

def norm1728(xy):
    x,y = xy
    return x**2 + y**2

def norm0order3(xy):
    x,y = xy
    return x**2 + x*y + y**2

def norm0order6(xy):
    x,y = xy
    return x**2 - x*y + y**2

def getapproxj0(p):
    r = find_gen_multgrp_orderm(3,p)
    if r is None:
        return None
    approx1 = (1,r)
    approx2 = next_approx(approx1,p)
    while norm0order3(approx1)!=p:
        if norm0order3(approx1)>p and norm0order3(approx2) < norm0order3(approx1):
            approx1 = approx2
            approx2 = next_approx(approx1,p)
        elif norm0order3(approx1)<p and norm0order6(approx2)>= p:
            x,y = approx1
            approx1 = (x,-y)
            approx2 = next_approx(approx1,p)
        else:
            return approx1
    return approx1
    
def split_2_1728(xy):
    x,y = xy
    if (x+y) % 2 != 0:
        return xy
    sm = (x + y)//2
    df = (x - y) //2
    if abs(sm) < abs(df):
        return (sm,df)
    else:
        return (df,sm)

def split_gen_1728(xy,l):
    if l % 4 != 1 or norm1728(xy) % l != 0:
        return xy
    x,y = xy
    vl = normsearch1728(l)
    if norm1728(vl) != l:
        return xy
    xl,yl = vl
    x1, y1 = xl*x + yl*y, xl*y - yl*x
    x2, y2 = xl*x - yl*y, xl*y + yl*x
    if x1 % l == 0 and y1 % l == 0:
        return (x1//l, y1//l)
    elif x2 % l == 0 and y2 % l == 0:
        return (x2//l, y2//l)
    else:
        return xy

def next_approx(xy,p):
    x,y = xy
    xnew1 = x*(p//y)
    ynew1 = p% y
    gxy1 = gcd(xnew1,ynew1)
    x1,y1 = (xnew1//gxy1,ynew1//gxy1)
    xnew2 = x*((p//y)-1)
    ynew2 = y *((p//y)-1)  - p
    gxy2 = gcd(xnew2,ynew2)
    x2,y2 = (xnew2//gxy2,ynew2//gxy2)
    if x2* y2 % p == 0:
        return (x1,y1)
    if x1* y1 % p == 0:
        return (x2,y2)
    if norm1728((x1,y1)) <= norm1728((x2,y2)):
        return (x1,y1)
    else:   
        return (x2,y2)  
    
def best_approx1728(v0,p):
    approx1 = v0
    approx2 = next_approx(approx1,p)
    while norm1728(approx2) < norm1728(approx1):
        approx1 = approx2
        approx2 = next_approx(approx1,p)
    return approx1

def normsearch1728(p):
    v0 = (1,find_gen_multgrp_orderm(4,p))
    v1= best_approx1728(v0,p)
    if norm1728(v1)==p:
        return v1
    else:
        x1,y1 = v1
        v2 = best_approx1728((x1,-y1),p)
        while norm1728(v2) < norm1728(v1) and norm1728(v2) != p:
            v1 = v2
            x1,y1 = v1
            v2 = best_approx1728((x1,-y1),p)
        nv2 = norm1728(v2)
        k = nv2//p
        while k %2 ==0:
            k = k //2
            v2 = split_2_1728(v2)
        if k > 1:
            pfk = primefact(k)
            for lf in pfk:
                v2 = split_gen_1728(v2,lf)
        return v2

In [210]:
find_gen_multgrp_orderm(4,421)

29

In [156]:
primes = primesBetween(15,10000)
prtest0 = {p: normsearch1728(p) for p in primes if p % 4 == 1}
failedsearch1m4 = [p for p in prtest0 if norm1728(prtest0[p]) != p]
failureindex1m4 = {p:norm1728(prtest0[p])//p for p in failedsearch1m4}

In [180]:
pr1m3test = {p: getapproxj0(p) for p in primes if p % 3 == 1}
failedsearch1m3 = [p for p in pr1m3test if norm0order3(pr1m3test[p]) != p]
failureindex1m3 = {p:norm0order3(pr1m3test[p]) for p in failedsearch1m3}

In [181]:
failureindex1m3[277]%277

17

In [169]:
{p for p in failureindex1m3 if failureindex1m3[p]%p!=0}

{277,
 367,
 397,
 547,
 613,
 631,
 673,
 691,
 733,
 751,
 883,
 919,
 997,
 1039,
 1069,
 1153,
 1171,
 1231,
 1327,
 1399,
 1423,
 1429,
 1549,
 1609,
 1621,
 1747,
 1753,
 1777,
 1789,
 1873,
 1933,
 1951,
 2161,
 2203,
 2239,
 2251,
 2281,
 2347,
 2377,
 2383,
 2521,
 2539,
 2683,
 2749,
 2791,
 2851,
 2917,
 3049,
 3067,
 3121,
 3169,
 3217,
 3229,
 3253,
 3259,
 3331,
 3463,
 3499,
 3511,
 3517,
 3607,
 3613,
 3631,
 3637,
 3673,
 3691,
 3739,
 3823,
 3877,
 3931,
 4021,
 4129,
 4159,
 4219,
 4243,
 4339,
 4363,
 4447,
 4507,
 4513,
 4567,
 4591,
 4621,
 4657,
 4663,
 4783,
 4813,
 4861,
 4909,
 4957,
 4987,
 5059,
 5077,
 5101,
 5197,
 5227,
 5407,
 5413,
 5431,
 5449,
 5563,
 5581,
 5623,
 5653,
 5689,
 5743,
 5821,
 5857,
 5869,
 5923,
 6043,
 6079,
 6151,
 6217,
 6229,
 6271,
 6301,
 6343,
 6361,
 6373,
 6379,
 6397,
 6451,
 6469,
 6529,
 6703,
 6763,
 6823,
 6829,
 6841,
 6907,
 7039,
 7057,
 7069,
 7129,
 7297,
 7309,
 7351,
 7369,
 7411,
 7417,
 7549,
 7561,
 7591,
 7603

In [211]:
rts1m4 = {p:find_gen_multgrp_orderm(4,p) for p in primes[:100] if p % 4 == 1}

In [ ]:
def next1728(xy,p):
    x,y = xy
    k = p//(abs(y))
    x1 = (x*k)%p
    x2 = (x*(k+1))%p
    y1 = p-k*abs(y)
    y2 = (k+1)*abs(y)-p
    v1 = (x1,y1)
    v2 = (x2,y2)
    n1 = norm1728(v1)
    n2 = norm1728(v2)
    if n1 == p or n1 <= n2:
        return v1
    else:
        return v2

In [222]:
oneshot1m4s = {}
p1m4x = {}
for p in rts1m4:
    r = rts1m4[p]
    if r**2+1 == p:
        oneshot1m4s[p]=(1,r)
    else:
        p1m4x[p]=[(1,r),next1728((1,r),p)]

In [223]:
p1m4x

{29: [(1, 12), (2, 5)],
 41: [(1, 9), (4, 5)],
 53: [(1, 23), (2, 7)],
 61: [(1, 11), (5, 6)],
 73: [(1, 27), (3, 8)],
 89: [(1, 34), (3, 13)],
 97: [(1, 22), (4, 9)],
 109: [(1, 33), (3, 10)],
 113: [(1, 15), (7, 8)],
 137: [(1, 37), (4, 11)],
 149: [(1, 44), (3, 17)],
 157: [(1, 28), (6, 11)],
 173: [(1, 80), (2, 13)],
 181: [(1, 19), (9, 10)],
 193: [(1, 81), (2, 31)],
 229: [(1, 107), (2, 15)],
 233: [(1, 89), (3, 34)],
 241: [(1, 64), (4, 15)],
 269: [(1, 82), (3, 23)],
 277: [(1, 60), (5, 23)],
 281: [(1, 53), (5, 16)],
 293: [(1, 138), (2, 17)],
 313: [(1, 25), (12, 13)],
 317: [(1, 114), (3, 25)],
 337: [(1, 148), (2, 41)],
 349: [(1, 136), (3, 59)],
 353: [(1, 42), (8, 17)],
 373: [(1, 104), (4, 43)],
 389: [(1, 115), (3, 44)],
 397: [(1, 63), (6, 19)],
 409: [(1, 143), (3, 20)],
 421: [(1, 29), (14, 15)],
 433: [(1, 179), (2, 75)],
 449: [(1, 67), (7, 20)],
 457: [(1, 109), (4, 21)],
 461: [(1, 48), (10, 19)],
 509: [(1, 208), (2, 93)],
 521: [(1, 235), (2, 51)],
 541: [(1, 5

In [227]:
{p:tuple([norm1728(xy)//p for xy in p1m4x[p]]) for p in p1m4x}

{29: (5, 1),
 41: (2, 1),
 53: (10, 1),
 61: (2, 1),
 73: (10, 1),
 89: (13, 2),
 97: (5, 1),
 109: (10, 1),
 113: (2, 1),
 137: (10, 1),
 149: (13, 2),
 157: (5, 1),
 173: (37, 1),
 181: (2, 1),
 193: (34, 5),
 229: (50, 1),
 233: (34, 5),
 241: (17, 1),
 269: (25, 2),
 277: (13, 2),
 281: (10, 1),
 293: (65, 1),
 313: (2, 1),
 317: (41, 2),
 337: (65, 5),
 349: (53, 10),
 353: (5, 1),
 373: (29, 5),
 389: (34, 5),
 397: (10, 1),
 409: (50, 1),
 421: (2, 1),
 433: (74, 13),
 449: (10, 1),
 457: (26, 1),
 461: (5, 1),
 509: (85, 17),
 521: (106, 5),
 541: (5, 1),
 557: (25, 2),
 569: (13, 2)}

In [36]:
r4p137=find_gen_multgrp_orderm(4,137)

In [233]:
primes[-1021]

1289

In [234]:
find_gen_multgrp_orderm(4,1289)

479

In [56]:
[(a,b) for a in range(1,14) for b in range(1,14) if a**2 + b**2 == 157]

[(6, 11), (11, 6)]

In [46]:
next_approx((7,15),137)

(63, 2)

In [73]:
split_odd((7,15))

(-8, 22)

In [74]:
norm1728((7,15))

274

In [76]:
norm1728((-4,11))

137

In [190]:
next_approx((1,29),421)

(14, 15)

In [191]:
14**2

196

In [192]:
15**2

225

In [235]:
next_approx((1,479),1289)

(2, 331)

In [198]:
589-407

182

In [199]:
pow(182,4,589)

163